# Columnformer Testing playground on ImageNet-100

## 1 - Getting started

TBD  

## 2 - Cloning the code

Clone your fork of the code from the original repository at : https://github.com/clane9/columnformers  if on colab

In [1]:
# Clone your repository
#!git clone https://github.com/IhabBendidi/columnformers.git

Install all dependencies as needed

In [22]:
!cd .. && python -m pip install .
!cd .. && git checkout -b depth
!pip3 install datasets
!pip3 install torch-summary
!pip3 install timm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Processing /import/pr_biofid/ihab/columnformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following versions that require a different python version: 0.1.0 Requires-Python >=3.9; 0.1.1 Requires-Python >=3.9
ERROR: Could not find a version that satisfies the requirement hf-argparser (from columnformers) (from versions: none)
ERROR: No matching distribution found for hf-argparser
fatal: Une branche nommée 'depth' existe déjà.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [23]:
!nvidia-smi

Wed Jan  3 11:48:24 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 510.60.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   31C    P0    35W / 250W |  12973MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   

Import packages

In [24]:
from torch.utils.data import DataLoader,Dataset
from datasets import load_dataset, load_from_disk
import torch.nn.functional as F
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import os
import tqdm

Define gpus to use. For now only on one gpu

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Importing the dataset from Hugging Face if not already available. If available, read from disk

In [26]:
if os.path.isdir("../../imagenet100.hf"):
    dataset = load_from_disk("../../imagenet100.hf")
else:
    dataset = load_dataset("clane9/imagenet-100")
    dataset.save_to_disk("../../imagenet100.hf")

#### Preparing datasets

In [27]:
from torch.utils.data import Subset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

Define the type of training you will have.

"full" : Using the full ImageNet-100 for training. Not recommended on Colab unless you have a paid account

"debug" : Using a small subset of ImageNet-100 containing 6k images in training, sampled in a balanced manner from the 100 classes, used for debugging and testing the architecture. (AKA Micro-ImageNet-100)

In [28]:
training_type = "full" # "debug" for a small 6k training subset of imagenet100, or "full" for the full imagenet100
batch_size = 512

Extracting Micro-ImageNet-100

In [29]:
import os
import numpy as np

# Function to load or extract labels
def load_or_extract_labels(dataset):
    labels = [dataset[i]['label'] for i in range(len(dataset))]
    return labels

In [30]:


# Define the custom dataset class
class CustomImageDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image']
        label = item['label']
        if self.transform:
            image = self.transform(image)
        return {'image': image, 'label': label}

# Updated transformations for the images
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),  # Convert all images to RGB
    transforms.ColorJitter(0.4,0.4,0.2,0.1),
    transforms.RandomResizedCrop(size=(128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),  # Convert all images to RGB
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Create instances of the custom dataset with transformations
train_dataset = CustomImageDataset(dataset['train'], transform=transform)
val_dataset = CustomImageDataset(dataset['validation'], transform=test_transform)

In [31]:
if training_type == "debug" :
    # Load or extract labels for training and test datasets
    train_labels = load_or_extract_labels(dataset['train'])
    test_labels = load_or_extract_labels(dataset['validation'])


In [32]:
if training_type == "debug" :
    # Stratified split
    sss_train = StratifiedShuffleSplit(n_splits=1, train_size=0.05, random_state=1)
    sss_test = StratifiedShuffleSplit(n_splits=1, test_size=0.10, random_state=0)
    
    train_indices, _ = next(sss_train.split(np.zeros(len(train_labels)), train_labels))
    _, test_indices = next(sss_test.split(np.zeros(len(test_labels)), test_labels))
    
    # Convert indices to Python integers
    train_indices = [int(i) for i in train_indices]
    test_indices = [int(i) for i in test_indices]
    
    # Create subset datasets
    train_subset = Subset(train_dataset, train_indices)
    test_subset = Subset(val_dataset, test_indices)  # Use val_dataset for test subset
    
    # Create data loaders for subsets
    train_subset_dataloader = DataLoader(train_subset, batch_size=batch_size, shuffle=True,num_workers=4)
    test_subset_dataloader = DataLoader(test_subset, batch_size=batch_size, shuffle=False,num_workers=4)

In [33]:
if training_type == "debug" :
    # Assuming train_subset and test_subset are your final subset datasets
    
    train_subset_size = len(train_subset)
    test_subset_size = len(test_subset)
    
    print(f"Size of Training Subset: {train_subset_size}")
    print(f"Size of Testing Subset: {test_subset_size}")


In [34]:
if training_type == "debug" :
    train_dataloader = train_subset_dataloader
    val_dataloader = test_subset_dataloader
elif training_type == "full" :
    # Create data loaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=4)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size,num_workers=4)

## Training the model

Importing the model architecture

In [35]:
import sys
import os

# Assuming your notebook is in the 'notebooks' directory
# and you want to import modules from the 'column' directory

# Get the current working directory
notebook_dir = os.getcwd()

# Get the parent directory (columnformers)
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [36]:
from columnformers.models.model_v1 import Columnformer, columnformer_v1_patch16_128
from columnformers.models import create_model
from columnformers.models.classification import ImageClassification

In [37]:
torch.cuda.empty_cache()

Initializing the model

In [38]:
columnformer = create_model("columnformer_v1_patch16_128")
model = ImageClassification(
        encoder=columnformer,
        img_size=128,
        patch_size=16,
        output_len=256,
        num_classes=100,
        global_pool="avg",
    ).to(device)

In [39]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [40]:
#scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.99)

In [41]:
from torchsummary import summary
summary(model)

Layer (type:depth-idx)                   Param #
├─Rearrange: 1-1                         --
├─Linear: 1-2                            295,296
├─ZeroPad2d: 1-3                         --
├─Columnformer: 1-4                      --
|    └─Sheet: 2-1                        --
|    |    └─ColumnNorm: 3-1              356,352
|    |    └─ColumnAttention: 3-2         23,051,520
|    |    └─ColumnNorm: 3-3              356,352
|    |    └─ColumnMlp: 3-4               22,836,224
├─GlobalAveragePool: 1-5                 --
├─LayerNorm: 1-6                         768
├─Dropout: 1-7                           --
├─Linear: 1-8                            38,500
Total params: 46,935,012
Trainable params: 46,935,012
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Rearrange: 1-1                         --
├─Linear: 1-2                            295,296
├─ZeroPad2d: 1-3                         --
├─Columnformer: 1-4                      --
|    └─Sheet: 2-1                        --
|    |    └─ColumnNorm: 3-1              356,352
|    |    └─ColumnAttention: 3-2         23,051,520
|    |    └─ColumnNorm: 3-3              356,352
|    |    └─ColumnMlp: 3-4               22,836,224
├─GlobalAveragePool: 1-5                 --
├─LayerNorm: 1-6                         768
├─Dropout: 1-7                           --
├─Linear: 1-8                            38,500
Total params: 46,935,012
Trainable params: 46,935,012
Non-trainable params: 0

Training the model for 100 epochs

In [42]:
for epoch in range(100):  # 100 epochs
    model.train()  # Set the model to training mode
    total_loss = 0
    correct = 0
    total = 0
    for batch in tqdm.tqdm(train_dataloader):
        optimizer.zero_grad()  # Zero the gradients

        # Move each tensor in the batch to the GPU
        images = batch["image"].to(device)
        labels = batch["label"].to(device)

        # Forward pass: Compute loss and state by passing the processed batch through the model
        loss, state = model({"image": images, "label": labels})

        outputs = state["output"]
        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Backward pass: Compute gradient of the loss with respect to model parameters
        loss.backward()

        # Perform a single optimization step (parameter update)
        optimizer.step()
        total_loss += loss.item()

        # Optional: Print loss, you might want to accumulate and print every few iterations
    #scheduler.step()
    train_accuracy = 100 * correct / total
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch}, Training Loss: {avg_loss}, Training Accuracy: {train_accuracy}%")


    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_dataloader:
            images = batch["image"].to(device)
            labels = batch["label"].to(device)

            loss, state = model({"image": images, "label": labels})
            val_loss += loss.item()
            outputs = state["output"]

            # Calculate validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Validation Epoch {epoch}, Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy}%")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 235/235 [02:38<00:00,  1.49it/s]

Epoch 0, Training Loss: 4.697567320884542, Training Accuracy: 4.277385496976804%


Validation Epoch 0, Validation Loss: 4.716021633148193, Validation Accuracy: 5.72%


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 235/235 [02:39<00:00,  1.47it/s]

Epoch 1, Training Loss: 4.463440000249984, Training Accuracy: 5.602268852349942%


...

Epoch 98, Training Loss: 2.8905743142391773, Training Accuracy: 33.158678277069455%


Validation Epoch 98, Validation Loss: 3.782669949531555, Validation Accuracy: 29.98%


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 235/235 [02:40<00:00,  1.46it/s]

Epoch 99, Training Loss: 2.8947095079624905, Training Accuracy: 33.085489491587445%


Validation Epoch 99, Validation Loss: 3.7545466661453246, Validation Accuracy: 29.48%
